In [1]:
import csv
import numpy as np
import pandas as pd
from collections import defaultdict
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import metrics # confusion matrix, MSE etc.

In [2]:
path_to_data = '/Users/t.z.cheng/Google_Drive/Research/Delaydoesmatter/real_exp/exp5_21CR01/results/session-5ffcf-6007b-data.csv'

In [3]:
## Read the data
df = pd.read_csv(path_to_data)

In [4]:
## Tasks & Conditions: filled
ftasks = ['f_maintaskTrials_2_l','f_maintaskTrials_2_s']
fconds = ['f300ms_delay_2_51','f300ms_delay_2_52','f300ms_delay_2_53','f300ms_delay_2_54','f300ms_delay_2_55','f300ms_delay_2_56',
        'f300ms_delay_2_81','f300ms_delay_2_82','f300ms_delay_2_83','f300ms_delay_2_84','f300ms_delay_2_85','f300ms_delay_2_86',
        'f300ms_delay_2_91','f300ms_delay_2_92','f300ms_delay_2_93','f300ms_delay_2_94','f300ms_delay_2_95','f300ms_delay_2_96']

In [6]:
## Tasks & Conditions: empty
etasks = ['e_maintaskTrials_2_l','e_maintaskTrials_2_s']
econds = ['e300ms_delay_2_51','e300ms_delay_2_52','e300ms_delay_2_53','e300ms_delay_2_54','e300ms_delay_2_55','e300ms_delay_2_56',
        'e300ms_delay_2_81','e300ms_delay_2_82','e300ms_delay_2_83','e300ms_delay_2_84','e300ms_delay_2_85','e300ms_delay_2_86',
        'e300ms_delay_2_91','e300ms_delay_2_92','e300ms_delay_2_93','e300ms_delay_2_94','e300ms_delay_2_95','e300ms_delay_2_96']

In [25]:
## Parameters
threshold = 0
conds = econds
tasks = etasks

In [26]:
subID = df['participant_id'].unique()
df_clean = df[(df['stimuli_presented'].isin(conds)) & (df['trial_template'].isin(tasks))]
env_noise = df[(df['response_name'] == 'survey_noise')]['response_value']

In [27]:
## Number of subjects, trials & conditions for each subjects for each trial type (empty vs filled)
n_subj = len(df_clean['participant_id'].unique())
n_trial = len(df_clean['trial_num'].unique())
n_conds = len(df_clean['stimuli_presented'].unique())
print('Participant_number:', n_subj,'Trial number:', n_trial,'Condition number:', n_conds, sep='\n')

Participant_number:
29
Trial number:
432
Condition number:
18


In [ ]:
## Look at the practice trials: change e to f
practice_tasks = ['f_practiceTrials_s','f_practiceTrials_l']
practice_conds = ['f300ms_delay_2_81','f300ms_delay_2_86']
df_practice = df[(df['stimuli_presented'].isin(practice_conds)) & (df['trial_template'].isin(practice_tasks))]

In [ ]:
practice_Accuracy = []
Acc_cond = []
for sub in subID:
    tmpdf = df_practice[df_practice['participant_id'] == sub]
    for cond in practice_conds:
        tmpAcc = np.mean(tmpdf[tmpdf['stimuli_presented'] == cond]['response_correct'])
        Acc_cond.append(tmpAcc)
    practice_Accuracy.append((Acc_cond[0] + Acc_cond[-1])/len(practice_conds))
    Acc_cond = []

In [ ]:
fpractice_Accuracy = pd.Series(practice_Accuracy, index=df_clean['participant_id'].unique())
cat = pd.concat([fpractice_Accuracy,epractice_Accuracy,pd.Series(np.array(env_noise),index=df_clean['participant_id'].unique())],axis = 1, ignore_index=False, names=['Filled','Empty'])

In [ ]:
practice_cat = cat

In [ ]:
practice_cat.describe()

In [ ]:
practice_cat

## Accuracy

In [28]:
Accuracy = []
Acc_cond = []
Easiest = []
Easiest_idx = [6,11] # e300ms_delay_2_81 and e300ms_delay_2_86
for sub in subID:
    tmpdf = df_clean[df_clean['participant_id'] == sub]
    for cond in conds:
        tmpAcc = np.mean(tmpdf[tmpdf['stimuli_presented'] == cond]['response_correct'])
        Acc_cond.append(tmpAcc)
    Easiest.append((Acc_cond[Easiest_idx[0]] + Acc_cond[Easiest_idx[-1]])/len(Easiest_idx)) # average accuracy of 81 and 86 
    Accuracy.append(Acc_cond)
    Acc_cond = []

In [29]:
## Filled duration: Outliers based on the accuracy of the easiest trials 
Acc_easiest = pd.Series(Easiest, index=df_clean['participant_id'].unique())
t = Acc_easiest[Acc_easiest >= threshold]
print("Subjects who have >=",threshold, "accuracy for the easiest trials:",len(t))

Subjects who have >= 0 accuracy for the easiest trials: 29


## Concatenate the filled and empty
Need to rerun the previous codes but change the tasks and conds 

In [14]:
#fAcc_easiest = Acc_easiest
fAcc_easiest = Acc_easiest
cat = pd.concat([fAcc_easiest,eAcc_easiest,pd.Series(np.array(env_noise),index=df_clean['participant_id'].unique())],axis = 1, ignore_index=False, names=['Filled','Empty'])
cat.describe()

NameError: name 'eAcc_easiest' is not defined

In [ ]:
## check environmental noise and accuracy of the easiest trials 
cat

In [ ]:
cat[cat[0]<0.5]

In [ ]:
cat[cat[1]<0.5]

In [31]:
## Exclude the outliers: change fAccuracy, fconds and fAcc_easiest to e-- to see the empty duration 
df_Accuracy = pd.DataFrame(data = Accuracy, index = subID, columns = conds)
Acc_final_sample = df_Accuracy[Acc_easiest >= threshold]
Acc_final_sample.head()
print('Final sample size:', len(Acc_final_sample))
# Acc_final_sample.describe()

Final sample size: 29


In [32]:
## Early (0:5), ontime(6:11), late (12:-1)
early = [np.arange(0,6,1)]
ontime = [np.arange(6,11,1)]
late = [np.arange(12,17,1)]
Acc_early = Acc_final_sample.iloc[:, early[0]].mean(axis = 1)
Acc_ontime = Acc_final_sample.iloc[:, ontime[0]].mean(axis = 1)
Acc_late = Acc_final_sample.iloc[:, late[0]].mean(axis = 1)
print("Accuracy")
print("Early:", np.mean(Acc_early))
print("Ontime:", np.mean(Acc_ontime))
print("Late:",np.mean(Acc_late))

Accuracy
Early: 0.5584291187739464
Ontime: 0.6063218390804599
Late: 0.5706896551724137


In [ ]:
## Visualization: bar plot
ACC_plot = pd.DataFrame({'Conditions': conds, 'acc': np.mean(Acc_final_sample, axis = 0)})
ax = ACC_plot.plot.bar(rot = 0)

## Proportion short

In [33]:
resp_short = []
resp_short = [True if d == 'Shorter' else False for d in df_clean['response_value']]
df_clean['resp_short'] = resp_short
PPS = []
PPS_cond = []
for sub in subID:
    tmpdf = df_clean[df_clean['participant_id'] == sub]
    for cond in conds:
        tmpPPS = np.mean(tmpdf[tmpdf['stimuli_presented'] == cond]['resp_short'])
        PPS_cond.append(tmpPPS)
    PPS.append(PPS_cond)
    PPS_cond = []

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
df_PPS = pd.DataFrame(data = PPS, index = subID, columns = conds)
PPS_final_sample = df_PPS[Acc_easiest >= threshold]
# PPS_final_sample.describe()

In [35]:
## Early (0:5), ontime(6:11), late (12:-1)
PPS_early = PPS_final_sample.iloc[:, early[0]].mean(axis = 1)
PPS_ontime = PPS_final_sample.iloc[:, ontime[0]].mean(axis = 1)
PPS_late = PPS_final_sample.iloc[:, late[0]].mean(axis = 1)
print("Proportion short")
print("Early:", np.mean(PPS_early))
print("Ontime:", np.mean(PPS_ontime))
print("Late:",np.mean(PPS_late))

Proportion short
Early: 0.7576628352490421
Ontime: 0.5063218390804598
Late: 0.38678160919540244


In [ ]:
## Visualization: bar plot
PPS_plot = pd.DataFrame({'Conditions': conds, 'PPS': np.mean(PPS_final_sample, axis = 0)})
ePPS_plot = PPS_plot
ax = PPS_plot.plot.bar(rot = 0, ylim = [0,1])

In [ ]:
PPS_plot.plot(rot = 0,lw=10)
plt.plot(np.transpose(PPS_final_sample))
plt.xticks(np.arange(0, len(conds), step=1),conds)  # Set label locations.
plt.show()